In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv(r'C:\Users\Rui\Desktop\RTAFall2022Projects\ad sites\ad sites.csv')
df

,URL,Category
0,http://30702100.com/,List of web with pictures
1,https://abouttechinfo.com/,List of web with pictures
2,http://www.allmedrcmsolutions.com/,List of web with pictures
3,https://appwyt.com/,List of web with pictures
4,http://www.b2bhg.com/,List of web with pictures
...,...,...
112,http://mg100.com.cn/,floating picture
113,https://www.afternic.com/forsale/bankoflincoln...,Go daddy
114,http://satelliteinternetaccess.net/,Go daddy
115,https://cloud-about.ru/,Comment section


In [3]:
pic = df[df['Category']=='Calendar/archives']
pic

,URL,Category
27,http://aboutcegonsoft.com/,Calendar/archives
28,http://www.artechstudios.com/,Calendar/archives
29,https://www.askbrien.com/,Calendar/archives
30,https://www.beisboldata.com/,Calendar/archives
31,https://betersoft.at.ua/,Calendar/archives
32,http://www.biocomputingassociates.com/,Calendar/archives
33,https://biotechcomms.com/,Calendar/archives
34,http://c-community.net/,Calendar/archives
35,https://www.cleantechrecruits.com/,Calendar/archives
36,https://www.communitymedia.net/,Calendar/archives


In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

In [7]:
all_html = {}
for index, rows in pic.iterrows():
    cur_domain = rows['URL']
    try:
        page = urlopen(cur_domain)
        soup = BeautifulSoup(page, 'html.parser')
        all_html[cur_domain] = soup
        print(cur_domain, '--done')
    except:
        print(cur_domain,'--error')

http://aboutcegonsoft.com/ --done
http://www.artechstudios.com/ --error
https://www.askbrien.com/ --error
https://www.beisboldata.com/ --done
https://betersoft.at.ua/ --done
http://www.biocomputingassociates.com/ --done
https://biotechcomms.com/ --done
http://c-community.net/ --done
https://www.cleantechrecruits.com/ --done
https://www.communitymedia.net/ --done
http://indiawebspace.net/ --error
http://learningtechnologies.zone/ --done


In [8]:
patterns = ['calendar']

In [9]:
for key, value in all_html.items():
    for pattern in patterns:
        if re.search(pattern, str(value)):
            print(key, '--found')
        else:
            print(key, '--not found')

http://aboutcegonsoft.com/ --found
https://www.beisboldata.com/ --found
https://betersoft.at.ua/ --not found
http://www.biocomputingassociates.com/ --found
https://biotechcomms.com/ --found
http://c-community.net/ --found
https://www.cleantechrecruits.com/ --not found
https://www.communitymedia.net/ --found
http://learningtechnologies.zone/ --found
